In [1]:
import pandas as pd

In [10]:
df = pd.read_csv(r"C:\Users\Rohan\Pictures\rohan\NLP\preprocessed_data.csv")

In [11]:
df.head()

,Genre,Poem
0,music,walk cloud wrap ancient symbolsw descend hill ...
1,music,"thick brushthey spend hottest part day , soak ..."
2,music,"storm gener . someth easi surrend , sit window..."
3,music,—after ana mendieta carri around matin star ? ...
4,music,aja sherrard 20the portent may memori . —walla...


In [12]:
df.shape

(841, 2)

In [13]:
df.duplicated().sum()


6

In [18]:
df.drop_duplicates( keep='first',inplace=True)

In [19]:
df.isnull().sum()

Genre    0
Poem     0
dtype: int64

In [20]:
df.dropna(inplace=True)

In [21]:
df.shape

(835, 2)

In [22]:
df.Genre.value_counts()

Genre
music          238
death          229
environment    227
affection      141
Name: count, dtype: int64

In [23]:
df['Genre'] = df['Genre'].replace({'affection': 3, 'environment':2 ,'music': 1, 'death': 0})
df['Genre'].value_counts()

Genre
1    238
0    229
2    227
3    141
Name: count, dtype: int64

In [24]:
df.head()

,Genre,Poem
0,1,walk cloud wrap ancient symbolsw descend hill ...
1,1,"thick brushthey spend hottest part day , soak ..."
2,1,"storm gener . someth easi surrend , sit window..."
3,1,—after ana mendieta carri around matin star ? ...
4,1,aja sherrard 20the portent may memori . —walla...


In [26]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Define a function for text preprocessing
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters, numbers, and punctuations
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Stemming (using Porter Stemmer)
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    
    # Join tokens back into a single string
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

# Apply the preprocessing function to the 'combined_text' column
df['Poem_preprocess_text'] = df['Poem'].apply(preprocess_text)

# Display the DataFrame with preprocessed text
df.head()


,Genre,Poem,Poem_preprocess_text
0,1,walk cloud wrap ancient symbolsw descend hill ...,walk cloud wrap ancient symbolsw descend hill ...
1,1,"thick brushthey spend hottest part day , soak ...",thick brushthey spend hottest part day soak ho...
2,1,"storm gener . someth easi surrend , sit window...",storm gener someth easi surrend sit window ste...
3,1,—after ana mendieta carri around matin star ? ...,ana mendieta carri around matin star hold fore...
4,1,aja sherrard 20the portent may memori . —walla...,aja sherrard portent may memori wallac stevens...


In [31]:
df.Poem[3]

'—after ana mendieta carri around matin star ? hold forest-fir one hand ? would wake radiat , shimmer , gleam lucero-light ? morn would measur wingspan idea take off— & night would'

In [32]:
df.Poem_preprocess_text[3]

'ana mendieta carri around matin star hold forestfir one hand would wake radiat shimmer gleam lucerolight morn would measur wingspan idea take night would'

In [34]:
preprocess_text('i am rohan barad , how are u ans the que withs')

'rohan barad u an que with'

In [111]:
df['Genre'].value_counts()

Genre
1    238
0    229
2    227
3    141
Name: count, dtype: int64

In [112]:
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

X = df.drop('Genre', axis=1)
y = df['Genre']
ros = RandomOverSampler(random_state=42)

X_resampled, y_resampled = ros.fit_resample(X, y)
df_resampled = pd.concat([X_resampled, y_resampled], axis=1)

print('Before oversampling:', Counter(y))
print('After oversampling:', Counter(y_resampled))

Before oversampling: Counter({1: 238, 0: 229, 2: 227, 3: 141})
After oversampling: Counter({1: 238, 0: 238, 3: 238, 2: 238})


In [113]:
df_resampled.duplicated().sum()

117

In [114]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

X = df_resampled['Poem_preprocess_text']
y = df_resampled['Genre']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize text data
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train a Multinomial Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train_vectorized, y_train)

# Predictions on the test set
y_pred = nb_model.predict(X_test_vectorized)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Display classification report
print(classification_report(y_test, y_pred))


Accuracy: 0.55
              precision    recall  f1-score   support

           0       0.54      0.48      0.51        46
           1       0.45      0.51      0.48        41
           2       0.59      0.39      0.47        49
           3       0.61      0.78      0.68        55

    accuracy                           0.55       191
   macro avg       0.55      0.54      0.53       191
weighted avg       0.55      0.55      0.54       191



In [ ]:
Accuracy: 0.44
              precision    recall  f1-score   support

           0       0.35      0.53      0.43        43
           1       0.39      0.51      0.44        45
           2       0.69      0.50      0.58        54
           3       0.25      0.04      0.07        25

    accuracy                           0.44       167
   macro avg       0.42      0.40      0.38       167
weighted avg       0.46      0.44      0.43       167

In [36]:
df.columns

Index(['Genre', 'Poem', 'Poem_preprocess_text'], dtype='object')

In [74]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import CountVectorizer

# # Assuming 'df' is your DataFrame with columns 'news_type' and 'combined_text'
# # Define X and y
# X = df['Poem_preprocess_text']
# y = df['Genre']

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# # Initialize CountVectorizer
# vectorizer = CountVectorizer()

# # Fit and transform the training data
# X_train_vectorized = vectorizer.fit_transform(X_train)
# 
# # Transform the test data
# X_test_vectorized = vectorizer.transform(X_test)

# # Display the shape of the vectorized data
# print("Shape of X_train_vectorized:", X_train_vectorized.shape)
# print("Shape of X_test_vectorized:", X_test_vectorized.shape)


In [120]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Train a Multinomial Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train_vectorized, y_train)

# Calculate training accuracy
train_accuracy = nb_model.score(X_train_vectorized, y_train)
print(f'Training Accuracy: {train_accuracy:.2f}')

# Predictions on the test set
y_pred = nb_model.predict(X_test_vectorized)

# Evaluate the model on the test set
test_accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {test_accuracy:.2f}')

# Display classification report
print(classification_report(y_test, y_pred))

Training Accuracy: 0.95
Test Accuracy: 0.55
              precision    recall  f1-score   support

           0       0.54      0.48      0.51        46
           1       0.45      0.51      0.48        41
           2       0.59      0.39      0.47        49
           3       0.61      0.78      0.68        55

    accuracy                           0.55       191
   macro avg       0.55      0.54      0.53       191
weighted avg       0.55      0.55      0.54       191



In [116]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
classifiers = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC()
}
for name, classifier in classifiers.items():
    classifier.fit(X_train_vectorized, y_train)
    y_pred = classifier.predict(X_test_vectorized)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {accuracy}")

Logistic Regression Accuracy: 0.5602094240837696
Decision Tree Accuracy: 0.4869109947643979
Random Forest Accuracy: 0.5497382198952879
SVM Accuracy: 0.5602094240837696


In [117]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the pipeline
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', LogisticRegression())
])

# Define the hyperparameters grid for tuning
param_grid = {
    'vectorizer__ngram_range': [(1, 1), (1, 2)],  # Uni-gram or Bi-gram
    'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],  # Inverse of regularization strength
    'classifier__penalty': ['l1', 'l2']  # Regularization penalty ('l1' for L1 regularization, 'l2' for L2 regularization)
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Get the best model
best_model = grid_search.best_estimator_

# Evaluate the best model on testing data
y_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("Testing Accuracy:", test_accuracy)


C:\Users\Rohan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
60 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Rohan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Rohan\anaconda3\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Rohan\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 475, in fit
    self._final_estimator.fi

Best Hyperparameters: {'classifier__C': 10, 'classifier__penalty': 'l2', 'vectorizer__ngram_range': (1, 2)}
Testing Accuracy: 0.5863874345549738


In [ ]:
# Training accuracy
train_accuracy = grid_search.best_score_
print("Training Accuracy:", train_accuracy)

In [40]:
df.columns

Index(['Genre', 'Poem', 'Poem_preprocess_text'], dtype='object')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences



tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(df_resampled['Poem_preprocess_text'])
X_sequences = tokenizer.texts_to_sequences(df_resampled['Poem_preprocess_text'])
X_padded = pad_sequences(X_sequences, maxlen=100)


y_labels = df_resampled['Genre']


In [ ]:


# ... (previous code for data preprocessing)

# Train-test split for DL
X_train_dl, X_test_dl, y_train_dl, y_test_dl = train_test_split(
    X_padded, y_labels, test_size=0.2, random_state=42
)

# Build a faster neural network
dl_model = Sequential()
dl_model.add(Embedding(input_dim=5000, output_dim=32, input_length=100))
dl_model.add(Flatten())
dl_model.add(BatchNormalization())
dl_model.add(Dense(64, activation='relu'))
dl_model.add(Dense(1, activation='sigmoid'))

# Use Adam optimizer with a lower learning rate
optimizer = Adam(learning_rate=0.001)

# Compile the model
dl_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Implement early stopping and model checkpoints
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

# Train the model
dl_model.fit(
    X_train_dl, y_train_dl,
    epochs=20, batch_size=32,
    validation_data=(X_test_dl, y_test_dl),
    callbacks=[early_stopping, model_checkpoint]
)


In [ ]:
eval_results = dl_model.evaluate(X_test_dl, y_test_dl)

# Extract accuracy from the evaluation results
accuracy = eval_results[1]
print("Test Accuracy:", accuracy)


In [56]:
eval_results = dl_model.evaluate(X_test_dl, y_test_dl)

# Extract accuracy from the evaluation results
test_accuracy = eval_results[1]
print("Test Accuracy:", test_accuracy)


history = dl_model.fit(
    X_train_dl, y_train_dl,
    epochs=20, batch_size=32,
    validation_data=(X_test_dl, y_test_dl),
    callbacks=[early_stopping, model_checkpoint])

# Extract training accuracy from the history object
train_accuracy = history.history['accuracy'][-1]
print("Training Accuracy:", train_accuracy)

6/6 [==============================] - 0s 5ms/step - loss: -19792.0664 - accuracy: 0.2695
Test Accuracy: 0.269461065530777
Epoch 1/20
21/21 [==============================] - 0s 19ms/step - loss: -34931.7383 - accuracy: 0.3174 - val_loss: -23489.1387 - val_accuracy: 0.2695
Epoch 2/20
21/21 [==============================] - 0s 16ms/step - loss: -39690.0742 - accuracy: 0.3189 - val_loss: -27421.6504 - val_accuracy: 0.2695
Epoch 3/20
21/21 [==============================] - 0s 14ms/step - loss: -45421.6016 - accuracy: 0.3189 - val_loss: -31751.4785 - val_accuracy: 0.2695
Epoch 4/20
21/21 [==============================] - 0s 16ms/step - loss: -51767.9648 - accuracy: 0.3174 - val_loss: -36628.9570 - val_accuracy: 0.2695
Epoch 5/20
21/21 [==============================] - 0s 14ms/step - loss: -58655.7852 - accuracy: 0.3069 - val_loss: -41883.8125 - val_accuracy: 0.2695
Epoch 6/20
21/21 [==============================] - 0s 14ms/step - loss: -64847.6953 - accuracy: 0.3263 - val_loss: -47922

In [62]:
df.columns

Index(['Genre', 'Poem', 'Poem_preprocess_text'], dtype='object')

In [63]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Poem_preprocess_text'])
sequences = tokenizer.texts_to_sequences(df['Poem_preprocess_text'])
max_sequence_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df['Genre'])
num_classes = len(label_encoder.classes_)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# Model Definition
embedding_dim = 100
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model Training
batch_size = 64
epochs = 10
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), verbose=2)

# Model Evaluation
score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]}')
print(f'Test accuracy: {score[1]}')


Epoch 1/10
11/11 - 10s - loss: 1.3789 - accuracy: 0.2904 - val_loss: 1.3693 - val_accuracy: 0.2695 - 10s/epoch - 871ms/step
Epoch 2/10
11/11 - 4s - loss: 1.3490 - accuracy: 0.3099 - val_loss: 1.3619 - val_accuracy: 0.3293 - 4s/epoch - 335ms/step
Epoch 3/10
11/11 - 3s - loss: 1.3036 - accuracy: 0.5659 - val_loss: 1.3587 - val_accuracy: 0.2934 - 3s/epoch - 309ms/step
Epoch 4/10
11/11 - 3s - loss: 1.1305 - accuracy: 0.6392 - val_loss: 1.3683 - val_accuracy: 0.3413 - 3s/epoch - 299ms/step
Epoch 5/10
11/11 - 3s - loss: 0.9249 - accuracy: 0.7964 - val_loss: 1.4366 - val_accuracy: 0.3772 - 3s/epoch - 317ms/step
Epoch 6/10
11/11 - 3s - loss: 0.6054 - accuracy: 0.8054 - val_loss: 1.6344 - val_accuracy: 0.4311 - 3s/epoch - 309ms/step
Epoch 7/10
11/11 - 3s - loss: 0.3907 - accuracy: 0.8997 - val_loss: 1.5059 - val_accuracy: 0.4132 - 3s/epoch - 300ms/step
Epoch 8/10
11/11 - 3s - loss: 0.2581 - accuracy: 0.9192 - val_loss: 1.8582 - val_accuracy: 0.4311 - 3s/epoch - 246ms/step
Epoch 9/10
11/11 - 3s 

In [65]:
from tensorflow.keras import layers

# Model Definition
embedding_dim = 200  # Increased embedding dimension
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(SpatialDropout1D(0.2))

# Adding multiple LSTM layers
model.add(LSTM(128, return_sequences=True, dropout=0.2))
model.add(LSTM(64, dropout=0.2))

# Adding Dense layers
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model Training
batch_size = 128
epochs = 20  # Increased epochs
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), verbose=2)

# Model Evaluation
score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]}')
print(f'Test accuracy: {score[1]}')


Epoch 1/20
6/6 - 13s - loss: 1.3835 - accuracy: 0.2740 - val_loss: 1.3760 - val_accuracy: 0.2695 - 13s/epoch - 2s/step
Epoch 2/20
6/6 - 2s - loss: 1.3705 - accuracy: 0.3159 - val_loss: 1.3641 - val_accuracy: 0.2934 - 2s/epoch - 389ms/step
Epoch 3/20
6/6 - 2s - loss: 1.3398 - accuracy: 0.3653 - val_loss: 1.3582 - val_accuracy: 0.2994 - 2s/epoch - 411ms/step
Epoch 4/20
6/6 - 3s - loss: 1.2990 - accuracy: 0.4476 - val_loss: 1.3562 - val_accuracy: 0.2754 - 3s/epoch - 477ms/step
Epoch 5/20
6/6 - 3s - loss: 1.1670 - accuracy: 0.4985 - val_loss: 1.3918 - val_accuracy: 0.2934 - 3s/epoch - 458ms/step
Epoch 6/20
6/6 - 3s - loss: 0.8695 - accuracy: 0.6781 - val_loss: 1.4212 - val_accuracy: 0.4371 - 3s/epoch - 432ms/step
Epoch 7/20
6/6 - 3s - loss: 0.6226 - accuracy: 0.8263 - val_loss: 1.6246 - val_accuracy: 0.3473 - 3s/epoch - 465ms/step
Epoch 8/20
6/6 - 3s - loss: 0.4819 - accuracy: 0.8608 - val_loss: 1.6354 - val_accuracy: 0.4251 - 3s/epoch - 450ms/step
Epoch 9/20
6/6 - 3s - loss: 0.3259 - accu

In [66]:
from tensorflow.keras import regularizers

# Model Definition
embedding_dim = 200
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(SpatialDropout1D(0.2))

# Adding multiple LSTM layers with dropout and recurrent dropout
model.add(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))

# Adding Dense layers with regularization
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(layers.Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model Training
batch_size = 128
epochs = 20
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), verbose=2)

# Model Evaluation
score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]}')
print(f'Test accuracy: {score[1]}')


Epoch 1/20
6/6 - 19s - loss: 2.0117 - accuracy: 0.2425 - val_loss: 1.9733 - val_accuracy: 0.2814 - 19s/epoch - 3s/step
Epoch 2/20
6/6 - 7s - loss: 1.9499 - accuracy: 0.3398 - val_loss: 1.9111 - val_accuracy: 0.2754 - 7s/epoch - 1s/step
Epoch 3/20
6/6 - 7s - loss: 1.8832 - accuracy: 0.3698 - val_loss: 1.8506 - val_accuracy: 0.2754 - 7s/epoch - 1s/step
Epoch 4/20
6/6 - 7s - loss: 1.8120 - accuracy: 0.3997 - val_loss: 1.8225 - val_accuracy: 0.3234 - 7s/epoch - 1s/step
Epoch 5/20
6/6 - 7s - loss: 1.6897 - accuracy: 0.5329 - val_loss: 1.7842 - val_accuracy: 0.3114 - 7s/epoch - 1s/step
Epoch 6/20
6/6 - 7s - loss: 1.4748 - accuracy: 0.5434 - val_loss: 1.7122 - val_accuracy: 0.4072 - 7s/epoch - 1s/step
Epoch 7/20
6/6 - 6s - loss: 1.3148 - accuracy: 0.6257 - val_loss: 1.6782 - val_accuracy: 0.3593 - 6s/epoch - 1s/step
Epoch 8/20
6/6 - 6s - loss: 1.1183 - accuracy: 0.7156 - val_loss: 1.8300 - val_accuracy: 0.3114 - 6s/epoch - 1s/step
Epoch 9/20
6/6 - 6s - loss: 0.8623 - accuracy: 0.8743 - val_lo

In [86]:
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=5000, output_dim=32, input_length=100))
lstm_model.add(LSTM(64, kernel_regularizer=regularizers.l2(0.01), return_sequences=True)) 
lstm_model.add(Dense(1, activation='sigmoid'))

# Compile the model
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Implement early stopping and model checkpoints
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_lstm_model.h5', save_best_only=True)

# Train the model
lstm_history = lstm_model.fit(
    X_train_dl, y_train_dl,
    epochs=10, batch_size=32,
    validation_data=(X_test_dl, y_test_dl),
    callbacks=[early_stopping, model_checkpoint]
)

# Evaluate the LSTM model on the test set
lstm_eval_results = lstm_model.evaluate(X_test_dl, y_test_dl)

# Extract accuracy from the evaluation results
lstm_test_accuracy = lstm_eval_results[1]
print("LSTM Test Accuracy:", lstm_test_accuracy)

# Extract training accuracy from the LSTM history object
lstm_train_accuracy = lstm_history.history['accuracy'][-1]
print("LSTM Training Accuracy:", lstm_train_accuracy)


Epoch 1/10
21/21 [==============================] - 8s 147ms/step - loss: 0.5563 - accuracy: 0.2943 - val_loss: -1.0142 - val_accuracy: 0.2695
Epoch 2/10
 2/21 [=>............................] - ETA: 0s - loss: -0.6554 - accuracy: 0.2188

C:\Users\Rohan\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


21/21 [==============================] - 1s 60ms/step - loss: -1.5831 - accuracy: 0.2889 - val_loss: -2.5986 - val_accuracy: 0.2695
Epoch 3/10
21/21 [==============================] - 1s 61ms/step - loss: -2.8268 - accuracy: 0.2889 - val_loss: -3.6412 - val_accuracy: 0.2695
Epoch 4/10
21/21 [==============================] - 1s 60ms/step - loss: -3.5783 - accuracy: 0.2889 - val_loss: -4.4032 - val_accuracy: 0.2695
Epoch 5/10
21/21 [==============================] - 1s 59ms/step - loss: -4.1771 - accuracy: 0.2889 - val_loss: -4.9173 - val_accuracy: 0.2695
Epoch 6/10
21/21 [==============================] - 1s 60ms/step - loss: -4.5659 - accuracy: 0.2889 - val_loss: -5.2875 - val_accuracy: 0.2695
Epoch 7/10
21/21 [==============================] - 1s 60ms/step - loss: -4.7552 - accuracy: 0.2889 - val_loss: -5.3259 - val_accuracy: 0.2695
Epoch 8/10
21/21 [==============================] - 1s 61ms/step - loss: -4.7842 - accuracy: 0.2889 - val_loss: -5.3490 - val_accuracy: 0.2695
Epoch 9/10

# LogisticRegression

In [133]:
LR = LogisticRegression(penalty='l2',random_state=21)
LR.fit(X_train_vectorized, y_train)

y_pred = LR.predict(X_test_vectorized)

# Calculate training accuracy
train_accuracy = nb_model.score(X_train_vectorized, y_train)
print(f'Training Accuracy: {train_accuracy:.2f}')

test_accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {test_accuracy:.2f}')

print(classification_report(y_test, y_pred))

Training Accuracy: 0.95
Test Accuracy: 0.56
              precision    recall  f1-score   support

           0       0.53      0.43      0.48        46
           1       0.43      0.61      0.51        41
           2       0.57      0.47      0.52        49
           3       0.71      0.71      0.71        55

    accuracy                           0.56       191
   macro avg       0.56      0.56      0.55       191
weighted avg       0.57      0.56      0.56       191



In [126]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Train a Multinomial Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train_vectorized, y_train)

# Calculate training accuracy
train_accuracy = nb_model.score(X_train_vectorized, y_train)
print(f'Training Accuracy: {train_accuracy:.2f}')

# Predictions on the test set
y_pred = nb_model.predict(X_test_vectorized)

# Evaluate the model on the test set
test_accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {test_accuracy:.2f}')

# Display classification report
print(classification_report(y_test, y_pred))

Training Accuracy: 0.95
Test Accuracy: 0.55
              precision    recall  f1-score   support

           0       0.54      0.48      0.51        46
           1       0.45      0.51      0.48        41
           2       0.59      0.39      0.47        49
           3       0.61      0.78      0.68        55

    accuracy                           0.55       191
   macro avg       0.55      0.54      0.53       191
weighted avg       0.55      0.55      0.54       191



# Predictions

In [134]:
df.sample(10)

,Genre,Poem,Poem_preprocess_text
609,3,", never except thirti second year ecstasi come...",never except thirti second year ecstasi come l...
606,3,nobodyuntil anoth man leavesa note wiper : lik...,nobodyuntil anoth man leavesa note wiper like ...
233,1,know pretendsto broken wing tolur predat away ...,know pretendsto broken wing tolur predat away ...
265,0,"second ago heart thump went thought , `` would...",second ago heart thump went thought would bad ...
598,3,"'s madg , madg men ? buri alic hair , ( ask ra...",madg madg men buri alic hair ask rain tell bea...
540,3,"sens world short , —long variou report , — lov...",sen world short long variou report love belov ...
729,2,"fuchsia funnel break outof crabappl tree , nei...",fuchsia funnel break outof crabappl tree neigh...
393,0,pretti girl . weather knock given lake wear sk...,pretti girl weather knock given lake wear skin...
535,3,door open saw thereand first time heard speak ...,door open saw thereand first time heard speak ...
559,3,"true love humbl , low-born thing , hath food s...",true love humbl lowborn thing hath food serv e...


In [151]:
Text = [df.Poem[606]]

In [152]:
# Example user input text
user_input = Text
df['Genre'] = df['Genre'].replace({'affection': 3, 'environment':2 ,'music': 1, 'death': 0})
user_input_vectorized = vectorizer.transform(user_input)

# Make predictions using the trained model
user_predictions = LR.predict(user_input_vectorized)

# Display the predictions
print("Predicted Genre:", user_predictions)

Predicted Genre: [3]


In [153]:
# Mapping genre labels to numeric values
label_mapping = {'affection': 3, 'environment': 2, 'music': 1, 'death': 0}

user_input = Text
user_input_vectorized = vectorizer.transform(user_input)

user_predictions = nb_model.predict(user_input_vectorized)

predicted_genre = [k for k, v in label_mapping.items() if v == user_predictions[0]][0]

print("Predicted Genre:", predicted_genre)

Predicted Genre: affection


# This model is not good becouse the data is small and the model is 

Training Accuracy: 0.95
Test Accuracy: 0.55
    
so means model is traing good but test accuracy is bad so this model is overfit 

In summary, the provided model is likely overfitting the training data,
as indicated by the large gap between the training and test accuracies. 
To address this, you may need to consider reducing the complexity of the model,
gathering more training data, or applying regularization techniques.